<a href="https://colab.research.google.com/github/mmcuervo/ExamMLcategorias/blob/main/01_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#En este archivo analisamos los datos del dataset

In [1]:
# 1) Importar librerías básicas para EDA
import pandas as pd              # Manejo de datos en DataFrames
import numpy as np               # Utilidades numéricas (vectores, matrices, NaN)
import matplotlib.pyplot as plt  # Gráficas base
import seaborn as sns            # Gráficas estadísticas (opcional, solo EDA)
from pathlib import Path         # Manejo seguro de rutas de archivos


In [6]:
#Llamamos al Dataset
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/AiLabData/Support_tickets.csv"
df = pd.read_csv(DATA_PATH, encoding="utf-8")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#Verificamos el archivo
print("Dimensiones del dataset:" )
df.info()
print(df.isnull().sum())
print("\nNúmero de filas duplicadas:", df.duplicated().sum())
df.head()

Dimensiones del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ticket_id               50000 non-null  int64  
 1   day_of_week             50000 non-null  object 
 2   day_of_week_num         50000 non-null  int64  
 3   company_id              50000 non-null  int64  
 4   company_size            50000 non-null  object 
 5   company_size_cat        50000 non-null  int64  
 6   industry                50000 non-null  object 
 7   industry_cat            50000 non-null  int64  
 8   customer_tier           50000 non-null  object 
 9   customer_tier_cat       50000 non-null  int64  
 10  org_users               50000 non-null  int64  
 11  region                  50000 non-null  object 
 12  region_cat              50000 non-null  int64  
 13  past_30d_tickets        50000 non-null  int64  
 14  past_90d_inci

,ticket_id,day_of_week,day_of_week_num,company_id,company_size,company_size_cat,industry,industry_cat,customer_tier,customer_tier_cat,...,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,customer_sentiment,customer_sentiment_cat,description_length,priority,priority_cat
0,1000000000,Wed,3,100015,Small,1,media,7,Basic,1,...,6,0,0,0,0,neutral,2,227,low,1
1,1000000001,Sat,6,100023,Small,1,healthcare,5,Basic,1,...,2,0,0,0,0,neutral,2,461,low,1
2,1000000002,Mon,1,100012,Small,1,gaming,4,Basic,1,...,0,0,0,0,1,positive,3,306,low,1
3,1000000003,Wed,3,100003,Small,1,media,7,Plus,2,...,16,0,0,0,1,neutral,2,363,medium,2
4,1000000004,Mon,1,100019,Small,1,ecommerce,2,Plus,2,...,6,0,0,0,0,neutral,2,442,low,1


In [20]:
#Cuento los diferentes niveles en las diferentes columnas categóricas
cols_cat=['day_of_week','company_size','industry','customer_tier','region','product_area',
          'booking_channel','reported_by_role','customer_sentiment','priority']
for col in cols_cat:
  print(f'Columna {col}:{df[col].nunique()} subcategorias')

for col in cols_cat:
  print(df[col].value_counts())

Columna day_of_week:7 subcategorias
Columna company_size:3 subcategorias
Columna industry:7 subcategorias
Columna customer_tier:3 subcategorias
Columna region:3 subcategorias
Columna product_area:6 subcategorias
Columna booking_channel:4 subcategorias
Columna reported_by_role:5 subcategorias
Columna customer_sentiment:3 subcategorias
Columna priority:3 subcategorias
day_of_week
Mon    11126
Tue     9415
Thu     9132
Wed     8998
Fri     8799
Sat     1265
Sun     1265
Name: count, dtype: int64
company_size
Small     18901
Medium    17158
Large     13941
Name: count, dtype: int64
industry
saas_b2b      13107
media         11931
ecommerce      7672
gaming         4935
fintech        4457
logistics      4163
healthcare     3735
Name: count, dtype: int64
customer_tier
Plus          21967
Basic         15696
Enterprise    12337
Name: count, dtype: int64
region
AMER    17103
EMEA    17065
APAC    15832
Name: count, dtype: int64
product_area
mobile           10003
analytics         9898
auth  

In [23]:
sns.set_theme(context="notebook", style="whitegrid")
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")

#Creamos una copia y trabajams sobre ella
df0=df.copy()

In [24]:
#2 Identificar columnas numéricas y separar banderas (0/1) para tratarlas luego como categóricas
num_cols_raw = df0.select_dtypes(include=np.number).columns.tolist()
binary_as_cat = [col for col in num_cols_raw if df0[col].isin([0, 1]).all()]
num_cols = [col for col in num_cols_raw if col not in binary_as_cat]

In [25]:
#3 Reporte rápido de qué quedó en cada grupo
print(f"Col numéricas (excluyendo binarias): {num_cols}")
print(f"Col bandera 0/1 como categóricas: {binary_as_cat}")

Col numéricas (excluyendo binarias): ['ticket_id', 'day_of_week_num', 'company_id', 'company_size_cat', 'industry_cat', 'customer_tier_cat', 'org_users', 'region_cat', 'past_30d_tickets', 'past_90d_incidents', 'product_area_cat', 'booking_channel_cat', 'reported_by_role_cat', 'customers_affected', 'error_rate_pct', 'downtime_min', 'customer_sentiment_cat', 'description_length', 'priority_cat']
Col bandera 0/1 como categóricas: ['payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook']


In [26]:
#4 Descriptivos con percentiles finos para detectar colas y outliers
if num_cols:
  desc=df0[num_cols].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).T
  display(desc.sort_index())

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
booking_channel_cat,"50,000.000",2.125,1.027,1.000,1.000,1.000,1.000,1.000,2.000,3.000,4.000,4.000,4.000,4.000
company_id,"50,000.000","100,013.037",7.265,"100,001.000","100,001.000","100,002.000","100,003.000","100,007.000","100,013.000","100,019.000","100,023.000","100,024.000","100,025.000","100,025.000"
company_size_cat,"50,000.000",1.901,0.804,1.000,1.000,1.000,1.000,1.000,2.000,3.000,3.000,3.000,3.000,3.000
customer_sentiment_cat,"50,000.000",1.834,0.607,0.000,0.000,1.000,1.000,2.000,2.000,2.000,2.000,3.000,3.000,3.000
customer_tier_cat,"50,000.000",1.933,0.746,1.000,1.000,1.000,1.000,1.000,2.000,2.000,3.000,3.000,3.000,3.000
customers_affected,"50,000.000",182.980,427.013,0.000,0.000,2.000,3.000,8.000,31.000,151.000,514.000,892.000,"2,074.080","5,757.000"
day_of_week_num,"50,000.000",3.078,1.610,1.000,1.000,1.000,1.000,2.000,3.000,4.000,5.000,6.000,7.000,7.000
description_length,"50,000.000",369.795,104.316,20.000,139.000,200.000,237.000,297.000,368.000,440.250,506.000,544.000,617.000,815.000
downtime_min,"50,000.000",11.116,17.294,0.000,0.000,0.000,0.000,0.000,0.000,17.000,34.000,46.000,76.000,196.000
error_rate_pct,"50,000.000",4.640,4.639,0.000,0.000,0.376,0.725,1.583,3.258,6.154,10.138,13.590,22.211,79.815
